IMPORTING ALL LIBRARIES:

In [15]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle
import pandas as pd
import numpy as np

LOADING TRAINED MODEL:

In [16]:
model = load_model('model.h5')

LOADING ENCODER AND SCALER:

In [17]:
with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender = pickle.load(file)
with open('onehot_encoding_geo.pkl', 'rb') as file:
    onehot_encoding_geo = pickle.load(file)
with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)


In [18]:
#Example input data:(for prediction)
input_data = {
    'CreditScore': 650,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 35,
    'Tenure': 7,
    'Balance': 86000.98,
    'NumOfProducts': 1,
    'HasCrCard': 1,
    'IsActiveMember': 0,
    'EstimatedSalary': 95181.12
}

In [23]:
#converting data into frame:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,650,France,Male,35,7,86000.98,1,1,0,95181.12


In [24]:
#Encoding gender categorical variables:
input_df['Gender']=label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,650,France,1,35,7,86000.98,1,1,0,95181.12


In [28]:
#Geography encoded data:
geo_encoded = onehot_encoding_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoding_geo.get_feature_names_out(['Geography']))
geo_encoded_df


/Users/pravallika/DL/ANN/venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [29]:
#concatenation of 2 data frame original and encoded one:
input_df=pd.concat([input_df.drop("Geography",axis=1),geo_encoded_df],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,650,1,35,7,86000.98,1,1,0,95181.12,1.0,0.0,0.0


In [30]:
## Scaling the input data
input_scaled=scaler.transform(input_df)
input_scaled

array([[-0.0206057 ,  0.91186722, -0.36939437,  0.69506988,  0.15596711,
        -0.91104337,  0.64946204, -1.03115165, -0.09531737,  1.00053348,
        -0.57776083, -0.57735027]])

In [31]:
#prediction-
model.predict(input_scaled)

1/1 [==============================] - 0s 108ms/step


array([[0.11124422]], dtype=float32)

In [32]:
prediction=model.predict(input_scaled)

1/1 [==============================] - 0s 28ms/step


In [33]:
prediction

array([[0.11124422]], dtype=float32)

In [35]:
prediction_proba = prediction[0][0]

In [36]:
prediction_proba

0.111244224

In [37]:
if prediction_proba > 0.5:
    print('Customer is likely to churn.')
else:
    print('Customer is not likely to churn.')

Customer is not likely to churn.
